In [ ]:
!git clone https://github.com/shaoanlu/face_toolbox_keras.git
!pip uninstall -y tensorflow
!pip install tensorflow-gpu==1.13.1
!pip install keras==2.2.4
%cd face_toolbox_keras

!gdown https://drive.google.com/uc?id=1H37LER8mRRI4q_nxpS3uQz3DcGHkTrNU
!mv lresnet100e_ir_keras.h5 ./models/verifier/insightface/lresnet100e_ir_keras.h5
!gdown https://drive.google.com/uc?id=18MyyXQIwhR5I6gzipYMiJ9ywgvFWQMvI
!mv backbone_ir50_ms1m_keras.h5 ./models/verifier/face_evoLVe_ir50/backbone_ir50_ms1m_keras.h5
!gdown https://drive.google.com/uc?id=1P_eQHU8bNJEsB6hHt_fnltOwQVKIfhiX
!mv backbone_ir50_asia_keras.h5 ./models/verifier/face_evoLVe_ir50/backbone_ir50_asia_keras.h5

In [ ]:
import warnings
import cv2
import numpy as np
import math
import os
import imageio
from matplotlib import pyplot as plt
from PIL import Image
from skimage import transform as trans
from models.detector import face_detector

In [ ]:
warnings.filterwarnings("ignore")
fd = face_detector.FaceAlignmentDetector(
    lmd_weights_path="./models/detector/FAN/2DFAN-4_keras.h5"# 2DFAN-4_keras.h5, 2DFAN-1_keras.h5
)

In [ ]:
class ImageClass():
    "Stores the paths to images for a given class"
    def __init__(self, name, image_paths):
        self.name = name
        self.image_paths = image_paths
  
    def __str__(self):
        return self.name + ', ' + str(len(self.image_paths)) + ' images'
  
    def __len__(self):
        return len(self.image_paths)

def get_dataset(path, has_class_directories=True):
    dataset = []
    path_exp = os.path.expanduser(path)
    classes = [path for path in os.listdir(path_exp) \
                    if os.path.isdir(os.path.join(path_exp, path))]
    classes.sort()
    nrof_classes = len(classes)
    for i in range(nrof_classes):
        class_name = classes[i]
        facedir = os.path.join(path_exp, class_name)
        image_paths = get_image_paths(facedir)
        dataset.append(ImageClass(class_name, image_paths))
  
    return dataset

def get_image_paths(facedir):
    image_paths = []
    if os.path.isdir(facedir):
        images = os.listdir(facedir)
        image_paths = [os.path.join(facedir,img) for img in images]
    return image_paths

def to_rgb(img):
    w, h = img.shape
    ret = np.empty((w, h, 3), dtype=np.uint8)
    ret[:, :, 0] = ret[:, :, 1] = ret[:, :, 2] = img
    return ret

def draw_bounding_boxes(image, bboxes, color=(255, 0, 0), stroke = 4):
    for bbox in bboxes:
      image = cv2.rectangle(image.copy(), (int(bbox[1]), int(bbox[0])), (int(bbox[3]), int(bbox[2])), color, stroke)
    return image

def ScaleRotateTranslate(image, angle, center = None, new_center = None, scale = None, resample=Image.BICUBIC):
    #image = Image.fromarray(image)
    angle = math.radians(angle) #convert from degree to radians
    if (scale is None) and (center is None):
        return image.rotate(angle=angle, resample=resample)
    nx,ny = x,y = center
    sx=sy=1.0
    if new_center:
        (nx,ny) = new_center
    if scale:
        (sx,sy) = (scale, scale)
    
    cosine = math.cos(angle)
    sine = math.sin(angle)

    a = cosine/sx
    b = sine/sx
    c = x-nx*a-ny*b
    d = -sine/sy
    e = cosine/sy
    f = y-nx*d-ny*e
    
    return image.transform(image.size, Image.AFFINE, (a,b,c,d,e,f), resample=resample)

def center_crop(img, bbox, new_width=None, new_height=None):        
    

    if new_width is None:
      new_width = min(width, height)

    if new_height is None:
      new_height = min(width, height)
      
    newx1, newx2, newy1, newy2 = bbox[1], bbox[3], bbox[0], bbox[2]
    if (new_width < new_height):
      newx1 = bbox[1] - int(np.ceil((new_height - new_width) / 2))
      newx2 = bbox[3] + int(np.ceil((new_height - new_width) / 2))
      if (newx1 < 0):
        temp1 = newx1
        newx1 = 0
        newx2 += temp1
    if (new_height < new_width):
      newy1 = bbox[0] - int(np.ceil((new_width - new_height) / 2))
      newy2 = bbox[2] + int(np.ceil((new_width - new_height) / 2))
      if (newy1 < 0):
        temp2 = newy1
        newy1 = 0
        newy2 += temp2
    center_cropped_img = img.crop((newx1, newy1, newx2, newy2))

    return center_cropped_img

def CropResizePIL(image, bbox, scale = None, resample=Image.LANCZOS):
    image = Image.fromarray(image)
    w, h = image.size
    w1, h1 = (bbox[3]-bbox[1], bbox[2] - bbox[0])
    if (scale is None):
      scale = (w, h)
    img = center_crop(image, bbox, w1, h1)
    #scaleX = scale[0] / w
    #scaleY = scale[1] / h
    #sy, sy = (scaleX, scaleY)
    #img = image.crop((bbox[1], bbox[0], bbox[3], bbox[2]))
    img = img.resize(scale, resample=resample)
    return img
def convert_landmarks_68_to_5_array(landmarks):
        left_eye = np.mean(landmarks[36:42], axis=0)
        right_eye = np.mean(landmarks[42:48], axis=0)
        nose_tip = landmarks[30]
        left_mouth = landmarks[48]
        right_mouth = landmarks[54]
        new_landmarks = [
            left_eye[0], left_eye[1],
            right_eye[0], right_eye[1],
            nose_tip[0], nose_tip[1],
            left_mouth[0], left_mouth[1] ,
            right_mouth[0], right_mouth[1]]
        return new_landmarks    
def make_landmarks_68_to_5(landmarks):
  new_landmarks = []
  for landmark in landmarks:
    new_landmark = convert_landmarks_68_to_5_array(landmark)
    #for item in new_landmark:
    new_landmarks.append(new_landmark)
  #new_landmarks = new_landmarks.flatten()
  #new_landmarks = np.array(new_landmarks)
  return new_landmarks

def rotateImage(img, landmark):
  eye_left = (landmark[0], landmark[1])
  eye_right = (landmark[2], landmark[3])
  dY = eye_right[1] - eye_left[1]
  dX = eye_right[0] - eye_left[0]
  angle = np.degrees(np.arctan2(dY, dX)) - 90
  #print(f"angle: {angle}")
  #test_image = Image.open("images/8.jpg")
  image_center = ((eye_left[0]+eye_right[0])/2,(eye_left[1]+eye_right[1])/2)
  imgCopy = Image.fromarray(img)
  rotatedImg = ScaleRotateTranslate(imgCopy,center=image_center,angle=angle)
  return rotatedImg
  
def isLeftEyeAndRightEyeOnSameLine(left_eye, right_eye):
    if (int(left_eye[0]) == int(right_eye[0])):
        return True
    return False

In [ ]:
output_dir = "/content/aligned/"
input_dir = "/content/train/"
if not os.path.exists(output_dir):
	os.makedirs(output_dir)
    
dataset = get_dataset(input_dir)

nrof_images_total = 0
nrof_successfully_aligned = 0
for cls in dataset:
    output_class_dir = os.path.join(output_dir, cls.name)
    if not os.path.exists(output_class_dir):
        os.makedirs(output_class_dir)
    for image_path in cls.image_paths:
        nrof_images_total += 1
        filename = os.path.splitext(os.path.split(image_path)[1])[0]
        output_filename = os.path.join(output_class_dir, filename+'.png')
        print(image_path)
        if not os.path.exists(output_filename):
            try:
                img = imageio.imread(image_path)
            except (IOError, ValueError, IndexError) as e:
                errorMessage = '{}: {}'.format(image_path, e)
                print(errorMessage)
            else:
                if img.ndim<2:
                    print('Unable to align "%s"' % image_path)
                    text_file.write('%s\n' % (output_filename))
                    continue
                if img.ndim == 2:
                    img = to_rgb(img)
                img = img[:,:,0:3]

                bounding_boxes_class, landmark_points = fd.detect_face(img, with_landmarks=True)
                
                bounding_boxes = np.array(bounding_boxes_class)
                points = np.array(make_landmarks_68_to_5(landmark_points))
                #print(points)
                nrof_faces = bounding_boxes.shape[0]
                if nrof_faces>0:
                    det = bounding_boxes[:,0:4]
                    det_points = points
                    #print(det_points)
                    det_arr = []
                    det_points_arr = []
                    img_size = np.asarray(img.shape)[0:2]
                    if nrof_faces>1:
                        for i in range(nrof_faces):
                                det_arr.append(np.squeeze(det[i]))
                                det_points_arr.append(np.squeeze(det_points[i]))
                    else:
                        det_arr.append(np.squeeze(det))
                        det_points_arr.append(np.squeeze(det_points))
                    '''print(f"det_points arr: {det_points_arr}")
                    print(f"det arr: {det_arr}")
                    input(f"Press any key to continue")'''
                    #for i, det in enumerate(det_arr):
                    ii = 0
                    for det, det_point in zip(det_arr, det_points_arr):
                        det = np.squeeze(det)
                        #print(det_point)
                        det_point = np.squeeze(det_point)
                        '''print(f"det_point: {det_point}")
                        print(f"det: {det}")
                        input(f"Press any key to continue")'''
                        bb = np.zeros(4, dtype=np.int32)
                        margin = 20
                        bb[0] = np.maximum(det[0]-margin/2, 0)
                        bb[1] = np.maximum(det[1]-margin/2, 0)
                        bb[2] = np.minimum(det[2]+margin/2, img_size[1])
                        bb[3] = np.minimum(det[3]+margin/2, img_size[0])
                        
                        bbox = np.array([bb[0], bb[1], bb[0]+bb[2], bb[1]+bb[3]])
                        #print(bbox)
                        #print(det_point)
                        landmark = np.array([det_point[0], det_point[1], det_point[2], det_point[3], det_point[4],
                                            det_point[5], det_point[6], det_point[7], det_point[8], det_point[9]])
                        #rotate image depends on eye center, then detect face on new image and crop + resize
                        print(landmark)
                        image_ii = rotateImage(img, landmark)
                        newImg = np.array(image_ii)
                        new_bboxes, new_landmarks = fd.detect_face(newImg, with_landmarks=True)
                        landmarks = np.array(make_landmarks_68_to_5(new_landmarks))
                        t = 0
                        for landmark_i in landmarks:
                            l_eye = (landmark_i[0], landmark_i[1])
                            r_eye = (landmark_i[2], landmark_i[3])
                            if isLeftEyeAndRightEyeOnSameLine(l_eye, r_eye):
                                ii = t    
                                break
                            t += 1
                            
                        new_bbox = new_bboxes[ii]
                        
                        imageFinal = CropResizePIL(newImg, new_bbox, scale=(112, 112))
                        nimg = np.array(imageFinal)
                        '''landmarks = np.array([landmarks["left_eye"][0], landmarks["right_eye"][0], landmarks["nose"][0], landmarks["mouth_left"][0], landmarks["mouth_right"][0],
                         landmarks["left_eye"][1], landmarks["right_eye"][1], landmarks["nose"][1], landmarks["mouth_left"][1], landmarks["mouth_right"][1]])'''
                        #landmarks = landmarks.reshape((2,5)).T
                        #print(landmarks)
                        #nimg = face_preprocess.preprocess(img, bbox, landmarks, image_size=f'{args.image_size},{args.image_size}')
                        #cv2.imshow("test", cv2.cvtColor(nimg, cv2.COLOR_RGB2BGR))
                        #cropped = img[bb[1]:bb[3],bb[0]:bb[2],:]
                        #scaled = skimage.transform.resize(cropped, (args.image_size, args.image_size))
                        nrof_successfully_aligned += 1
                        filename_base, file_extension = os.path.splitext(output_filename)
                        output_filename_n = "{}_{}{}".format(filename_base, ii, file_extension)
                           
                        #scaled = img_as_ubyte(scaled)
                        #imageio.imwrite(output_filename_n, scaled)
                        imageio.imwrite(output_filename_n, nimg)
                        #text_file.write('%s %d %d %d %d\n' % (output_filename_n, bb[0], bb[1], bb[2], bb[3]))
                        ii += 1
                        #cv2.waitKey(0)
                        #cv2.destroyAllWindows()
                else:
                    print('Unable to align "%s"' % image_path)
                    text_file.write('%s\n' % (output_filename))
						
print('Total number of images: %d' % nrof_images_total)
print('Number of successfully aligned images: %d' % nrof_successfully_aligned)